In [4]:
# CSVファイルに記載したフォルダ・PDFファイル名を読み込み、
# PDFファイルのテキストをページ単位でCSVファイルに格納

import pandas as pd
import MeCab
import fitz
import re
import sys
sys.path.append('/work_dir') 
import talknize_module_20240909 as tk
import csv
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

In [5]:
# CSVを読み込む関数
def load_csv(file_path):
    """
    CSVを読み込み、DataFrameを返す
    file_path: CSVファイルのパス
    """
    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: CSV file not found at {file_path}")
        return None
    except pd.errors.EmptyDataError:
        print(f"Error: CSV file at {file_path} is empty or invalid")
        return None
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

In [6]:
# ファイルを処理する関数
def process_file(file_path):
    """
    個々のPDFファイルをページ単位でテキスト化し、同一フォルダ内にCSVファイルとして格納
    
    """
    if not os.path.exists(file_path):
        print(f"Error: File does not exist - {file_path}")
        return
    
    try:
        doc = fitz.open(file_path) 
        
        #ページ単位でテキストをリストに格納
        #page_textにはインデックスは付与されていない

        page_text=[]
        for page in doc:
            text = page.get_text() # プレーンテキストを取得
            #if text != '' :
                #無駄な改行を削除の上で'。'で改行、一文の範囲を明確にする
            text = re.sub(r'。','。\n', re.sub(r'\n','', text))
                #リストに格納
            page_text.append(text)

        #リストをcsvファイルに保存（PDFと拡張子のみ相違）
        df_txt = pd.DataFrame(page_text)
        csv_path = re.sub('.pdf', '', file_path) + ".csv"
        df_txt.to_csv(csv_path, index=True, encoding='utf-8_sig')

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")


In [7]:
# メインルーチン

def main():
    # CSVファイルのパス
    csv_path = 'data/IR_Source_241211.csv'  # CSVのパスを指定

    # CSVを読み込む
    df = load_csv(csv_path)
    if df is None:
        print("Failed to load CSV. Exiting program.")
        return

    # 必要な列名を指定
    folder_col = '格納フォルダ'
    file_col = '統合報告書PDF'

    # CSVの列が期待通りか確認
    if folder_col not in df.columns or file_col not in df.columns:
        print(f"Error: CSV file must contain '{folder_col}' and '{file_col}' columns")
        return

    # 処理対象のファイルパスをリストに収集
    file_paths = []
    for index, row in df.iterrows():
        folder_name = 'ir/2025/' + row[folder_col]
        file_name = row[file_col]

        # パスが有効かチェック
        if pd.isna(folder_name) or pd.isna(file_name):
            print(f"Skipping row {index}: Folder or File is missing")
            continue

        # ファイルパスを構築
        file_path = os.path.join(folder_name, file_name)
        file_paths.append(file_path)

    # 並列処理
    results = []
    max_workers = 4  # 並列で動作するスレッド数
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_file = {executor.submit(process_file, file_path): file_path for file_path in file_paths}
        for future in as_completed(future_to_file):
            result = future.result()
            results.append(result)

    # 処理結果をログ出力
    print("Processing completed. Summary:")
    for result in results:
        print(result)

if __name__ == "__main__":
    main()

#順に企業コード・企業名・PDFファイル名を取り込み


Processing completed. Summary:
None
None
None
